##JOINS CON SPARK DIEGO RDZ 163036


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_timestamp

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
from google.colab import files
uploaded = files.upload()

Saving customer_profile_dataset.csv to customer_profile_dataset.csv


In [4]:
customer = spark.read.csv('customer_profile_dataset.csv', header=True)
customer.show(5)

+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+
|customer_id|first_name|last_name|gender|      date_of_birth|               email|phone_number|        signup_date|      address|       city|state|zip_code|
+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+
|          1|    Robert|    Smith|Female|1994-06-14 21:40:27|jane.davis1@mail.com|634-106-4981|2016-10-16 17:23:25| 8465 Main St|San Antonio|   CA|   35566|
|          2|     Emily|   Garcia|Female|1989-09-21 17:56:31|robert.williams2@...|386-635-5998|2021-04-04 14:24:06|  305 Main St|   New York|   AZ|   23187|
|          3|   Jessica|    Brown|  Male|1984-01-21 21:43:13|emily.davis3@mail...|627-341-5213|2018-04-22 04:51:57|  5725 Oak St|    Chicago|   AZ|   99188|
|          4|   Michael|    Brown|  Male|1986-02-06 13:09:

In [5]:
from google.colab import files
uploaded = files.upload()

Saving products_dataset.csv to products_dataset.csv


In [6]:
products = spark.read.csv('products_dataset.csv', header=True)
products.show(5)

+----------+------------+--------+--------------+------+--------------------+
|product_id|product_name|category|price_per_unit| brand| product_description|
+----------+------------+--------+--------------+------+--------------------+
|         1|      Butter|   Dairy|         28.58|BrandB|Description for Rice|
|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|         3|        Milk|   Meats|         26.52|BrandE|Description for B...|
|         4|      Banana|  Grains|         26.12|BrandB|Description for A...|
|         5|        Rice|  Fruits|         21.94|BrandD|Description for B...|
+----------+------------+--------+--------------+------+--------------------+
only showing top 5 rows



In [7]:
from google.colab import files
uploaded = files.upload()

Saving purchase_history_dataset.csv to purchase_history_dataset.csv


In [8]:
purchase = spark.read.csv('purchase_history_dataset.csv', header=True)
purchase.show(5)

+-----------+-----------+----------+-------------------+--------+------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|
+-----------+-----------+----------+-------------------+--------+------------------+
|          1|          1|        42|2018-04-15 14:08:01|       3| 37.64207365077783|
|          2|          1|       138|2022-07-10 23:33:47|       4| 70.24710587172727|
|          3|          1|       403|2021-12-31 03:53:33|       3| 89.16889585975464|
|          4|          1|       193|2017-01-14 01:25:11|       2| 59.70505931112876|
|          5|          1|        26|2018-04-06 11:01:06|       3|101.77886387225126|
+-----------+-----------+----------+-------------------+--------+------------------+
only showing top 5 rows



##Responde, ¿cuántos clientes llamados "Robert" (nota cómo hay Males y Females), compraron algún producto lácteo (Dairy) en 2022 ?

In [34]:
# Filtrar clientes llamados "Robert"
roberts = customer.filter(customer.first_name == "Robert")

# Filtrar productos de la categoría "Dairy"
dairy_products = products.filter(products.category == "Dairy")

# Realizar los joins para vincular las tablas

purchase_customers = purchase.join(roberts, purchase.customer_id == roberts.customer_id, how='inner')
purchase_dairy_roberts = purchase_customers.join(dairy_products, purchase_customers.product_id == dairy_products.product_id, how='inner')
purchase_dairy_roberts.show()
purchase_dairy_roberts.count()

# Calcular la cantidad de clientes llamados "Robert" que compraron productos lácteos en 2022

purchase_dairy_roberts = purchase.withColumn('purchase_date', to_timestamp(col('purchase_date'), 'yyyy-MM-dd HH:mm:ss'))\
                                 .filter((col('purchase_date') >= lit("2022-01-01 00:00:00")) & (col('purchase_date') <= lit("2022-12-31 23:59:59")))
purchase_dairy_roberts.count()




+-----------+-----------+----------+-------------------+--------+------------------+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+----------+------------+--------+--------------+------+--------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|customer_id|first_name|last_name|gender|      date_of_birth|               email|phone_number|        signup_date|      address|       city|state|zip_code|product_id|product_name|category|price_per_unit| brand| product_description|
+-----------+-----------+----------+-------------------+--------+------------------+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+----------+------------+--------+--------------+------+--------------------+
|         52|          5|       234|2021-05-04 02:15

2434

##Eres empleado de BrandB. ¿En cuáles ciudades has vendido una mayor cantidad? (total_amount)

In [37]:
from pyspark.sql.functions import sum

# Filtrar las compras relacionadas con BrandB
brandb_sales = purchase.join(products, purchase.product_id == products.product_id, how='inner')\
                       .filter(col('brand') == 'BrandB')

# Unir tablas
brandb_city_sales = brandb_sales.join(customer, brandb_sales.customer_id == customer.customer_id, how='inner')

city_sales = brandb_city_sales.groupBy("city").agg(sum("total_amount").alias("total_sales"))
#Ordenar
top_cities = city_sales.orderBy(col("total_sales").desc())
top_cities.show()


+------------+------------------+
|        city|       total_sales|
+------------+------------------+
|     Chicago|29557.642903425036|
|     Houston| 28586.38258782695|
|     Phoenix| 28041.96419969516|
|Philadelphia|27492.074313737838|
|    New York|27246.218068437967|
| San Antonio|25221.051797835673|
| Los Angeles|23819.529899740137|
+------------+------------------+



##¿De cuánto es la mayor cantidad (quantity) que ha sido comprado por algún hombre O cuyo producto sea pan (Bread)?

In [41]:
from pyspark.sql.functions import col, lit, max

# Filtrar compras realizadas por hombres
male_purchases = purchase.join(customer, purchase.customer_id == customer.customer_id, how='inner')\
                         .filter(col('gender') == 'Male')\
                         .select("quantity")  # Selecciona solo la columna necesaria

# Filtrar compras cuyo producto sea pan (Bread)
bread_purchases = purchase.join(products, purchase.product_id == products.product_id, how='inner')\
                          .filter(col('product_name') == 'Bread')\
                          .select("quantity")  # Selecciona solo la columna necesaria

# Combinar (Hombres O Producto Pan)
combined_purchases = male_purchases.union(bread_purchases)

# Encontrar la mayor con (quantity)
max_quantity = combined_purchases.agg(max("quantity").alias("max_quantity"))
max_quantity.show()


+------------+
|max_quantity|
+------------+
|           5|
+------------+

